# EBAC - Regressão II - regressão múltipla

## Tarefa I

#### Previsão de renda

Vamos trabalhar com a base 'previsao_de_renda.csv', que é a base do seu próximo projeto. Vamos usar os recursos que vimos até aqui nesta base.

|variavel|descrição|
|-|-|
|data_ref                | Data de referência de coleta das variáveis |
|index                   | Código de identificação do cliente|
|sexo                    | Sexo do cliente|
|posse_de_veiculo        | Indica se o cliente possui veículo|
|posse_de_imovel         | Indica se o cliente possui imóvel|
|qtd_filhos              | Quantidade de filhos do cliente|
|tipo_renda              | Tipo de renda do cliente|
|educacao                | Grau de instrução do cliente|
|estado_civil            | Estado civil do cliente|
|tipo_residencia         | Tipo de residência do cliente (própria, alugada etc)|
|idade                   | Idade do cliente|
|tempo_emprego           | Tempo no emprego atual|
|qt_pessoas_residencia   | Quantidade de pessoas que moram na residência|
|renda                   | Renda em reais|

In [1]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
import statsmodels.api as sm
import patsy

In [2]:
df = pd.read_csv('previsao_de_renda.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             15000 non-null  int64  
 1   data_ref               15000 non-null  object 
 2   id_cliente             15000 non-null  int64  
 3   sexo                   15000 non-null  object 
 4   posse_de_veiculo       15000 non-null  bool   
 5   posse_de_imovel        15000 non-null  bool   
 6   qtd_filhos             15000 non-null  int64  
 7   tipo_renda             15000 non-null  object 
 8   educacao               15000 non-null  object 
 9   estado_civil           15000 non-null  object 
 10  tipo_residencia        15000 non-null  object 
 11  idade                  15000 non-null  int64  
 12  tempo_emprego          12427 non-null  float64
 13  qt_pessoas_residencia  15000 non-null  float64
 14  renda                  15000 non-null  float64
dtypes:

1. Ajuste um modelo para prever log(renda) considerando todas as covariáveis disponíveis.
    - Utilizando os recursos do Patsy, coloque as variáveis qualitativas como *dummies*.
    - Mantenha sempre a categoria mais frequente como casela de referência
    - Avalie os parâmetros e veja se parecem fazer sentido prático.  


2. Remova a variável menos significante e analise:
    - Observe os indicadores que vimos, e avalie se o modelo melhorou ou piorou na sua opinião.
    - Observe os parâmetros e veja se algum se alterou muito.  


3. Siga removendo as variáveis menos significantes, sempre que o *p-value* for menor que 5%. Compare o modelo final com o inicial. Observe os indicadores e conclua se o modelo parece melhor. 
    

In [4]:
# removendo variáveis desnecessárias
df.drop(columns=['Unnamed: 0', 'data_ref', 'id_cliente'], axis=1, inplace=True)
df.head()

,sexo,posse_de_veiculo,posse_de_imovel,qtd_filhos,tipo_renda,educacao,estado_civil,tipo_residencia,idade,tempo_emprego,qt_pessoas_residencia,renda
0,F,False,True,0,Empresário,Secundário,Solteiro,Casa,26,6.602740,1.0,8060.34
1,M,True,True,0,Assalariado,Superior completo,Casado,Casa,28,7.183562,2.0,1852.15
2,F,True,True,0,Empresário,Superior completo,Casado,Casa,35,0.838356,2.0,2253.89
3,F,False,True,1,Servidor público,Superior completo,Casado,Casa,30,4.846575,3.0,6600.77
4,M,True,False,0,Assalariado,Secundário,Solteiro,Governamental,33,4.293151,1.0,6475.97


In [5]:
# Verificando as modas de cada coluna do dataframe
df.mode()

,sexo,posse_de_veiculo,posse_de_imovel,qtd_filhos,tipo_renda,educacao,estado_civil,tipo_residencia,idade,tempo_emprego,qt_pessoas_residencia,renda
0,F,False,True,0.0,Assalariado,Secundário,Casado,Casa,40.0,4.216438,2.0,728.96
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1272.04
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2999.59
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4234.94
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5402.44
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9826.31
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,43433.94


In [6]:
# 1. Ajuste um modelo para prever log(renda) considerando todas as covariáveis disponíveis.
#   Utilizando os recursos do Patsy, coloque as variáveis qualitativas como dummies.
#   Mantenha sempre a categoria mais frequente como casela de referência
#   Avalie os parâmetros e veja se parecem fazer sentido prático.

y, X = patsy.dmatrices('''np.log(renda) ~ C(sexo, Treatment('F')) + C(posse_de_veiculo, Treatment(0)) + 
                                  C(posse_de_imovel, Treatment(1)) + qtd_filhos + C(tipo_renda, Treatment('Assalariado')) +
                                  C(educacao, Treatment('Secundário')) + C(estado_civil, Treatment('Casado')) + 
                                  C(tipo_residencia, Treatment('Casa')) + idade + tempo_emprego + qt_pessoas_residencia''', 
                       data = df, return_type='dataframe')
X.head()

,Intercept,"C(sexo, Treatment('F'))[T.M]","C(posse_de_veiculo, Treatment(0))[T.True]","C(posse_de_imovel, Treatment(1))[T.False]","C(tipo_renda, Treatment('Assalariado'))[T.Bolsista]","C(tipo_renda, Treatment('Assalariado'))[T.Empresário]","C(tipo_renda, Treatment('Assalariado'))[T.Pensionista]","C(tipo_renda, Treatment('Assalariado'))[T.Servidor público]","C(educacao, Treatment('Secundário'))[T.Primário]","C(educacao, Treatment('Secundário'))[T.Pós graduação]",...,"C(estado_civil, Treatment('Casado'))[T.Viúvo]","C(tipo_residencia, Treatment('Casa'))[T.Aluguel]","C(tipo_residencia, Treatment('Casa'))[T.Com os pais]","C(tipo_residencia, Treatment('Casa'))[T.Comunitário]","C(tipo_residencia, Treatment('Casa'))[T.Estúdio]","C(tipo_residencia, Treatment('Casa'))[T.Governamental]",qtd_filhos,idade,tempo_emprego,qt_pessoas_residencia
0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,26.0,6.602740,1.0
1,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,28.0,7.183562,2.0
2,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,35.0,0.838356,2.0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,30.0,4.846575,3.0
4,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,33.0,4.293151,1.0


In [7]:
# Treinando e mostrando o sumário do modelo
sm.OLS(y, X).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.357
Model:                            OLS   Adj. R-squared:                  0.356
Method:                 Least Squares   F-statistic:                     287.5
Date:                Wed, 27 Sep 2023   Prob (F-statistic):               0.00
Time:                        13:30:11   Log-Likelihood:                -13568.
No. Observations:               12427   AIC:                         2.719e+04
Df Residuals:                   12402   BIC:                         2.737e+04
Df Model:                          24                                         
Covariance Type:            nonrobust                                         
===============================================================================================================================
                                                                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------------------------------
Intercept                                                       6.6093      0.219     30.214      0.000       6.181       7.038
C(sexo, Treatment('F'))[T.M]                                    0.7874      0.015     53.723      0.000       0.759       0.816
C(posse_de_veiculo, Treatment(0))[T.True]                       0.0441      0.014      3.119      0.002       0.016       0.072
C(posse_de_imovel, Treatment(1))[T.False]                      -0.0829      0.014     -5.926      0.000      -0.110      -0.055
C(tipo_renda, Treatment('Assalariado'))[T.Bolsista]             0.2209      0.241      0.916      0.360      -0.252       0.694
C(tipo_renda, Treatment('Assalariado'))[T.Empresário]           0.1551      0.015     10.387      0.000       0.126       0.184
C(tipo_renda, Treatment('Assalariado'))[T.Pensionista]         -0.3087      0.241     -1.280      0.201      -0.782       0.164
C(tipo_renda, Treatment('Assalariado'))[T.Servidor público]     0.0576      0.022      2.591      0.010       0.014       0.101
C(educacao, Treatment('Secundário'))[T.Primário]                0.0141      0.072      0.196      0.844      -0.127       0.155
C(educacao, Treatment('Secundário'))[T.Pós graduação]           0.1212      0.142      0.853      0.394      -0.157       0.400
C(educacao, Treatment('Secundário'))[T.Superior completo]       0.1079      0.014      7.763      0.000       0.081       0.135
C(educacao, Treatment('Secundário'))[T.Superior incompleto]    -0.0290      0.032     -0.900      0.368      -0.092       0.034
C(estado_civil, Treatment('Casado'))[T.Separado]                0.3265      0.111      2.935      0.003       0.108       0.545
C(estado_civil, Treatment('Casado'))[T.Solteiro]                0.2672      0.109      2.453      0.014       0.054       0.481
C(estado_civil, Treatment('Casado'))[T.União]                  -0.0343      0.025     -1.361      0.174      -0.084       0.015
C(estado_civil, Treatment('Casado'))[T.Viúvo]                   0.3689      0.116      3.191      0.001       0.142       0.596
C(tipo_residencia, Treatment('Casa'))[T.Aluguel]                0.0442      0.054      0.815      0.415      -0.062       0.150
C(tipo_residencia, Treatment('Casa'))[T.Com os pais]            0.0184      0.029      0.625      0.532      -0.039       0.076
C(tipo_residencia, Treatment('Casa'))[T.Comunitário]           -0.0754      0.091     -0.829      0.407      -0.254       0.103
C(tipo_residencia, Treatment('Casa'))[T.Estúdio]                0.1107      0.084      1.319      0.187      -0.054       0.275
C(tipo_residencia, Treatment('Casa'))[T.Governamental]         -0.0129      0.039     -0.331      0.741      -0.089       0.064
qtd_filhos 

- O modelo apresenta um R-squared de 0.256 e um Adj. R-square de 	0.25.
- Um valor muito grande de AIC de 2.576e+05.
- Os desvios-padrão e o intervalo de confiança estão em uma amplitude muito alta.
- Vário parâmetros estão com com p-value acima de 0.05.
- Este modelo não está representando os dados de forma razoável.5

In [8]:
# 2. Remova a variável menos significante e analise:
# Observe os indicadores que vimos, e avalie se o modelo melhorou ou piorou na sua opinião.
# Observe os parâmetros e veja se algum se alterou muito.

# A variável menos significativa é C(educacao, Treatment('Secundário'))[T.Primário]	que possui p-value de 0.844
X.drop(columns=["C(educacao, Treatment('Secundário'))[T.Primário]"], axis=1, inplace=True)
X.head()

,Intercept,"C(sexo, Treatment('F'))[T.M]","C(posse_de_veiculo, Treatment(0))[T.True]","C(posse_de_imovel, Treatment(1))[T.False]","C(tipo_renda, Treatment('Assalariado'))[T.Bolsista]","C(tipo_renda, Treatment('Assalariado'))[T.Empresário]","C(tipo_renda, Treatment('Assalariado'))[T.Pensionista]","C(tipo_renda, Treatment('Assalariado'))[T.Servidor público]","C(educacao, Treatment('Secundário'))[T.Pós graduação]","C(educacao, Treatment('Secundário'))[T.Superior completo]",...,"C(estado_civil, Treatment('Casado'))[T.Viúvo]","C(tipo_residencia, Treatment('Casa'))[T.Aluguel]","C(tipo_residencia, Treatment('Casa'))[T.Com os pais]","C(tipo_residencia, Treatment('Casa'))[T.Comunitário]","C(tipo_residencia, Treatment('Casa'))[T.Estúdio]","C(tipo_residencia, Treatment('Casa'))[T.Governamental]",qtd_filhos,idade,tempo_emprego,qt_pessoas_residencia
0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,26.0,6.602740,1.0
1,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,28.0,7.183562,2.0
2,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,35.0,0.838356,2.0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,30.0,4.846575,3.0
4,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,33.0,4.293151,1.0


In [9]:
# Treinando e mostrando o sumário do modelo sem a variável menos significativa
sm.OLS(y, X).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.357
Model:                            OLS   Adj. R-squared:                  0.356
Method:                 Least Squares   F-statistic:                     300.0
Date:                Wed, 27 Sep 2023   Prob (F-statistic):               0.00
Time:                        13:30:12   Log-Likelihood:                -13568.
No. Observations:               12427   AIC:                         2.718e+04
Df Residuals:                   12403   BIC:                         2.736e+04
Df Model:                          23                                         
Covariance Type:            nonrobust                                         
===============================================================================================================================
                                                                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------------------------------
Intercept                                                       6.6098      0.219     30.219      0.000       6.181       7.039
C(sexo, Treatment('F'))[T.M]                                    0.7875      0.015     53.740      0.000       0.759       0.816
C(posse_de_veiculo, Treatment(0))[T.True]                       0.0441      0.014      3.119      0.002       0.016       0.072
C(posse_de_imovel, Treatment(1))[T.False]                      -0.0829      0.014     -5.929      0.000      -0.110      -0.055
C(tipo_renda, Treatment('Assalariado'))[T.Bolsista]             0.2209      0.241      0.916      0.360      -0.252       0.694
C(tipo_renda, Treatment('Assalariado'))[T.Empresário]           0.1551      0.015     10.386      0.000       0.126       0.184
C(tipo_renda, Treatment('Assalariado'))[T.Pensionista]         -0.3087      0.241     -1.280      0.201      -0.782       0.164
C(tipo_renda, Treatment('Assalariado'))[T.Servidor público]     0.0575      0.022      2.589      0.010       0.014       0.101
C(educacao, Treatment('Secundário'))[T.Pós graduação]           0.1210      0.142      0.852      0.394      -0.158       0.400
C(educacao, Treatment('Secundário'))[T.Superior completo]       0.1077      0.014      7.771      0.000       0.081       0.135
C(educacao, Treatment('Secundário'))[T.Superior incompleto]    -0.0293      0.032     -0.908      0.364      -0.092       0.034
C(estado_civil, Treatment('Casado'))[T.Separado]                0.3265      0.111      2.935      0.003       0.108       0.545
C(estado_civil, Treatment('Casado'))[T.Solteiro]                0.2671      0.109      2.453      0.014       0.054       0.481
C(estado_civil, Treatment('Casado'))[T.União]                  -0.0342      0.025     -1.359      0.174      -0.084       0.015
C(estado_civil, Treatment('Casado'))[T.Viúvo]                   0.3692      0.116      3.194      0.001       0.143       0.596
C(tipo_residencia, Treatment('Casa'))[T.Aluguel]                0.0442      0.054      0.816      0.414      -0.062       0.150
C(tipo_residencia, Treatment('Casa'))[T.Com os pais]            0.0185      0.029      0.629      0.530      -0.039       0.076
C(tipo_residencia, Treatment('Casa'))[T.Comunitário]           -0.0742      0.091     -0.817      0.414      -0.252       0.104
C(tipo_residencia, Treatment('Casa'))[T.Estúdio]                0.1106      0.084      1.318      0.187      -0.054       0.275
C(tipo_residencia, Treatment('Casa'))[T.Governamental]         -0.0127      0.039     -0.326      0.744      -0.089       0.064
qtd_filhos                                                     -0.2659      0.108     -2.463      0.014      -0.477      -0.054
idade      

- O novo modelo apresenta o R-squared = 0.357 e o 	Adj. R-square = 0.356. Os mesmos do modelo anterior. Assim como o mesmo AIC, indicando que não houve melhora com a remoção do parâmetro menos relevante.
- Porém existem no modelo diversos outros parâmetros não significantes para o modelo. 5

In [10]:
# 3. Siga removendo as variáveis menos significantes, sempre que o p-value for menor que 5%. 
# Compare o modelo final com o inicial.
# Observe os indicadores e conclua se o modelo parece melhor.

# Função para remover variáveis menos significantes e imprimir o sumário do modelo
def remove_variavel_menos_significante(modelo, X, y):
    print('Modelo inicial com todas as variáveis\n', modelo.summary(),'\n\n\n')
    while modelo.pvalues.max() > 0.05:
        variavel_menos_significante = modelo.pvalues.idxmax()
        X.drop(variavel_menos_significante, axis=1, inplace=True)
        modelo = sm.OLS(y, X).fit()
        print('Removendo a variável variavel menos significante: ', variavel_menos_significante, '\n', modelo.summary(), '\n\n\n')

# Calcula a matriz
formula = '''np.log(renda) ~ 1 + C(sexo, Treatment('F')) + C(posse_de_veiculo, Treatment(0)) + 
                             C(posse_de_imovel, Treatment(1)) + qtd_filhos + C(tipo_renda, Treatment('Assalariado')) +
                             C(educacao, Treatment('Secundário')) + C(estado_civil, Treatment('Casado')) + 
                             C(tipo_residencia, Treatment('Casa')) + idade + tempo_emprego + qt_pessoas_residencia'''
y_2, X_2 = patsy.dmatrices(formula, 
                           data = df, 
                           return_type='dataframe')

# Treina o primeiro modelo
modelo_2 = sm.OLS(y_2, X_2).fit()

# Treina os modelos seguintes removendo as variáveis com p-values > 0.005 e exibi o sumário de cada modelo
remove_variavel_menos_significante(modelo_2, X_2, y_2)


Modelo inicial com todas as variáveis
                             OLS Regression Results                            
Dep. Variable:          np.log(renda)   R-squared:                       0.357
Model:                            OLS   Adj. R-squared:                  0.356
Method:                 Least Squares   F-statistic:                     287.5
Date:                Wed, 27 Sep 2023   Prob (F-statistic):               0.00
Time:                        13:30:12   Log-Likelihood:                -13568.
No. Observations:               12427   AIC:                         2.719e+04
Df Residuals:                   12402   BIC:                         2.737e+04
Df Model:                          24                                         
Covariance Type:            nonrobust                                         
                                                                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------

- Não houve alteração significativa das métricas de avaliação do modelo com a remoção das variáveis menos relevantes.
- Porém um modelo com uma quantidade menor de variáveis é menos complexo e menos sujeito a overfiting